# Equity Index Trend Following Strategy

The following code is used for feature engineering and performance back-testing

In [2]:
import pandas as pd
import numpy as np
from scipy.stats import kurtosis
from scipy.stats import skew
from scipy.stats import norm

In [3]:
EQ = pd.read_csv("TSMOM_EQ.csv",parse_dates=True)
EQ['Date'] = pd.to_datetime(EQ['Date'], dayfirst=True)
EQ['month'] = EQ['Date'].astype(str).str[:7]
#EQ = EQ.set_index('month').drop('Date',axis=1)
EQ['total'] = EQ['TSMOM_EQ']

In [4]:
EQdf = EQ.copy().drop('total',axis=1)

for t in range(0,len(EQdf)):
    if EQdf.loc[t,'TSMOM_EQ'] > 0:
        EQdf.loc[t,'total'] = 1
    elif EQdf.loc[t,'TSMOM_EQ'] < 0:
        EQdf.loc[t,'total'] = 0
    else:
        EQdf.loc[t,'total'] = np.nan #zero % monthly return is given NaN

# Macro Data

In [5]:
indicators = ["US_CPI","US_PMI","US_Unemp"]
pct_change_indicators = ["US_CPI"]

In [6]:
US_CPI = pd.read_csv("Features/CSV/US Macro Data.csv",usecols=[1,2],skiprows=[0,1,2,3,4,5],names=["US_CPI","Date"])
US_PMI = pd.read_csv("Features/CSV/US Macro Data.csv",usecols=[7,8],skiprows=[0,1,2,3,4,5],names=["US_PMI","Date"])
US_Unemp = pd.read_csv("Features/CSV/US Macro Data.csv",usecols=[11,12],skiprows=[0,1,2,3,4,5],names=["US_Unemp","Date"])

In [7]:
#Set date axis
US_CPI['Date'] = pd.to_datetime(US_CPI['Date'], dayfirst=True)
US_CPI['month'] = US_CPI['Date'].astype(str).str[:7]
US_CPI = US_CPI.set_index('month').drop('Date',axis=1)

US_PMI['Date'] = pd.to_datetime(US_PMI['Date'], dayfirst=True)
US_PMI['month'] = US_PMI['Date'].astype(str).str[:7]
US_PMI = US_PMI.set_index('month').drop('Date',axis=1)

US_Unemp['Date'] = pd.to_datetime(US_Unemp['Date'], dayfirst=True)
US_Unemp['month'] = US_Unemp['Date'].astype(str).str[:7]
US_Unemp = US_Unemp.set_index('month').drop('Date',axis=1)

US_Unemp['US_Unemp'] = US_Unemp['US_Unemp'].shift(-1)
US_PMI['US_PMI'] = US_PMI['US_PMI'].shift(-1)
US_CPI['US_CPI'] = US_CPI['US_CPI'].shift(-1)

In [8]:
#Merging macro indicators to portfolio returns
EQdf = EQdf.set_index(EQdf['month']).drop('month',axis=1)
EQdf = EQdf.merge(US_CPI,how='left',on='month')
EQdf = EQdf.merge(US_PMI,how='left',on='month')
EQdf = EQdf.merge(US_Unemp,how='left',on='month')

In [9]:
#Manipulate Features
EQdf['US_CPI'] = EQdf['US_CPI'].pct_change()
EQdf['US_Unemp_change'] = EQdf['US_Unemp'].pct_change()
EQdf['US_PMI'] = EQdf['US_PMI']-50

# Surprises in Macroeconomic Data

In [10]:
Surprise_Indices_Names = ["Citi US Inflation Surprise","Citi EUR Inflation Surprise","Citi EM Inflation Surprise","Citi Global Inflation Surprise", "Citi APAC Inflation Surprise","Citi BRIC Inflation Surprise","Citi G10 Inflation Surprise","Citi US Economic Surprise","Citi EUR Economic Surprise","Citi EM Economic Surprise","Citi Global Economic Surprise","Citi APAC Economic Surprise","Citi BRIC Economic Surprise","Citi G10 Economic Surprise"]
parse_names = ["Date","Citi US Inflation Surprise","Citi EUR Inflation Surprise","Citi EM Inflation Surprise","Citi Global Inflation Surprise", "Citi APAC Inflation Surprise","Citi BRIC Inflation Surprise","Citi G10 Inflation Surprise","Citi US Economic Surprise","Citi EUR Economic Surprise","Citi EM Economic Surprise","Citi Global Economic Surprise","Citi APAC Economic Surprise","Citi BRIC Economic Surprise","Citi G10 Economic Surprise"]

In [11]:
Surprise_Indices = pd.read_csv("Features/CSV/Citi Surprise Indices.csv",skiprows=[0,1,2,3,4,5],usecols=[0,1,3,4,5,6,7,8,9,10,11,12,13,14,15],names=parse_names)

In [12]:
#is the surprise from last month same direction as this month? i would have thought this would be useful but apparently not
#for i in Surprise_Indices_Names:
 #   for t in range(0,len(Surprise_Indices)-1):
  #      if Surprise_Indices.loc[t+1, '{}'.format(i)] > 0 and Surprise_Indices.loc[t, '{}'.format(i)] > 0:
   #         Surprise_Indices.loc[t+1,'{}_TrendCont'.format(i)] = 1
    #    elif Surprise_Indices.loc[t+1, '{}'.format(i)] < 0 and Surprise_Indices.loc[t, '{}'.format(i)] < 0:
     #       Surprise_Indices.loc[t+1,'{}_TrendCont'.format(i)] = 1
      #  else:
       #     Surprise_Indices.loc[t+1,'{}_TrendCont'.format(i)] = 0
        
#try using absolute difference - again did not help performance
#for i in Surprise_Indices_Names:
 #   for t in range(0,len(Surprise_Indices)-1):
  #      Surprise_Indices.loc[t+1, '{}_TrendCont'.format(i)] =  Surprise_Indices.loc[t+1, '{}'.format(i)] - Surprise_Indices.loc[t, '{}'.format(i)]

for i in Surprise_Indices_Names:
    #Surprise_Indices['{}'.format(i)] = abs(Surprise_Indices['{}'.format(i)]) #absolute value not valuable
    Surprise_Indices['{}'.format(i)] = Surprise_Indices['{}'.format(i)].shift(1)
    #Surprise_Indices['{}_TrendCont'.format(i)] = Surprise_Indices['{}_TrendCont'.format(i)].shift(1)
    
Surprise_Indices['Date'] = pd.to_datetime(Surprise_Indices['Date'], dayfirst=True)
Surprise_Indices['month'] = Surprise_Indices['Date'].astype(str).str[:7]
Surprise_Indices = Surprise_Indices.set_index('month').drop('Date',axis=1)

In [13]:
EQdf = EQdf.merge(Surprise_Indices,how='left',on='month')

# VIX Index

In [14]:
VIX = pd.read_csv("Features/CSV/VIX.csv",skiprows=[0,1,2,4,5])

In [15]:
VIX['Date'] = pd.to_datetime(VIX['Date'], dayfirst=True)
VIX['month'] = VIX['Date'].astype(str).str[:7]
VIX = VIX.set_index('month').drop('Date',axis=1)

In [16]:
VIX['VIX Index'] = VIX['VIX Index'].shift(1)
VIX['VIX_pctchange'] = VIX['VIX Index'].pct_change()

In [17]:
EQdf = EQdf.merge(VIX,how='left',on='month')

# Equity Indices


In [18]:
EQ_Indices_Names = ["MSCI_World","MSCI_EM","MSCI_APAC","MSCI_EUR","MSCI_EAFE","MSCI_ACWI"]

In [19]:
EQ_Indices = pd.read_csv("Features/CSV/Equity Indices (MSCI).csv",usecols=[0,1,2,3,4,5,6],skiprows=[0,1,2,3,4,5],names=["Date","MSCI_World","MSCI_EM","MSCI_APAC","MSCI_EUR","MSCI_EAFE","MSCI_ACWI"])

In [20]:
EQ_Indices['Date'] = pd.to_datetime(EQ_Indices['Date'], dayfirst=True)
EQ_Indices['month'] = EQ_Indices['Date'].astype(str).str[:7]
EQ_Indices = EQ_Indices.set_index('month').drop('Date',axis=1)

In [21]:
#Shift factors back 1 month so no look ahead bias
for i in EQ_Indices_Names:
    EQ_Indices['{}'.format(i)] = EQ_Indices['{}'.format(i)].shift(1)
for i in EQ_Indices_Names:
    EQ_Indices['{}'.format(i)] = EQ_Indices['{}'.format(i)].pct_change()

In [22]:
EQdf = EQdf.merge(EQ_Indices,how='left',on='month')

# Government Bond Indices

In [23]:
Bond_Indices_Names = ["US_Bond_RoR","US_Bond_Yield","UK_Bond_RoR","UK_Bond_Yield","GER_Bond_RoR","GER_Bond_Yield","JP_Bond_RoR","JP_Bond_Yield","AU_Bond_RoR","AU_Bond_Yield"]
Bond_Indices_RoR = ["US_Bond_RoR","UK_Bond_RoR","GER_Bond_RoR","JP_Bond_RoR","AU_Bond_RoR"]
Bond_Indices_Yield = ["US_Bond_Yield","UK_Bond_Yield","GER_Bond_Yield","JP_Bond_Yield","AU_Bond_Yield"]

In [24]:
Bond_Indices = pd.read_csv("Features/CSV/Government Bonds (Aggregate).csv",usecols=[0,1,2,3,4,5,6,7,8,9,10],skiprows=[0,1,2,3,4,5],names=["Date","US_Bond_RoR","US_Bond_Yield","UK_Bond_RoR","UK_Bond_Yield","GER_Bond_RoR","GER_Bond_Yield","JP_Bond_RoR","JP_Bond_Yield","AU_Bond_RoR","AU_Bond_Yield"])

In [25]:
Bond_Indices['Date'] = pd.to_datetime(Bond_Indices['Date'], dayfirst=True)
Bond_Indices['month'] = Bond_Indices['Date'].astype(str).str[:7]
Bond_Indices = Bond_Indices.set_index('month').drop('Date',axis=1)

In [26]:
#Shift factors back 1 month so no look ahead bias
for i in Bond_Indices_Names:
    Bond_Indices['{}'.format(i)] = Bond_Indices['{}'.format(i)].shift(1)
for i in Bond_Indices_RoR: #total monthly return
    Bond_Indices['{}'.format(i)] = Bond_Indices['{}'.format(i)].pct_change()
#for i in Bond_Indices_Yield: #% change in yield
 #   Bond_Indices['{}_pctchange'.format(i)] = Bond_Indices['{}'.format(i)].pct_change()
for i in Bond_Indices_Yield: #% change in yield
    Bond_Indices['{}'.format(i)] = Bond_Indices['{}'.format(i)].pct_change()

In [27]:
EQdf = EQdf.merge(Bond_Indices,how='left',on='month')

# Yield Curve Shape

In [28]:
Bond_Yields = pd.read_csv("Features/CSV/Bond Yields.csv",usecols=[0,1,2,3,4,5,6,7,8,9],skiprows=[0,1,2,4,5])

Bond_Yields['Date'] = pd.to_datetime(Bond_Yields['Date'], dayfirst=True)
Bond_Yields['month'] = Bond_Yields['Date'].astype(str).str[:7]
Bond_Yields = Bond_Yields.set_index('month').drop('Date',axis=1)

#Shift factors back 1 month so no look ahead bias
for i in list(Bond_Yields.columns):
    Bond_Yields['{}'.format(i)] = Bond_Yields['{}'.format(i)].shift(1)

Bond_Yields['US_YieldCurve'] = Bond_Yields['GT10 Govt'] - Bond_Yields['GT2 Govt']
Bond_Yields['UK_YieldCurve'] = Bond_Yields['GTGBP10Y Govt'] - Bond_Yields['GTGBP2Y Govt']
Bond_Yields['JP_YieldCurve'] = Bond_Yields['GTJPY10Y Govt'] - Bond_Yields['GTJPY2Y Govt']
Bond_Yields['EU_YieldCurve'] = Bond_Yields['GTEUR10Y Govt'] - Bond_Yields['GTEUR2Y Govt']

YieldCurve = Bond_Yields.iloc[:,-4:]

EQdf = EQdf.merge(YieldCurve,how='left',on='month')

# Output to R

In [29]:
EQdf_output = EQdf.copy().drop('Date',axis=1).drop('TSMOM_EQ',axis=1)

In [30]:
#Remove NAs
#EQdf = EQdf.dropna(subset=['total']) #remove NAs only based on a certain column
EQdf_output.dropna().to_csv('EQ_Portfolio Returns & Macro Data.csv') #remove all NAs

Now Run R Code

# Updated Positioning

In [31]:
def quadratic(x):
    if x > 0.5:
        y = 2*((x-0.5)**2)+1
        return y
    elif x < 0.5:
        y = -2*((x-0.5)**2)+1
        return y
    else:
        return 1
    
def quad_downside(x):
    if x>=0.5:
        return 1
    elif x<0.5:
        y = -2*((x-0.5)**2)+1
        return y
    else:
        return 1
        
def quad_upside(x): #use when model has high precision
    if x>=0.5:
        y = 2*((x-0.5)**2)+1
        return y
    elif x<0.5:
        return 1
    else:
        return 1
        
def linear(x):
    y= 2*x
    return y

def linear_downside(x):
    if x>=0.5:
        return 1
    elif x<0.5:
        return x

def ReLU(x): #use for high precision models
    if x>=0.5:
        y = 2*x
        return y
    elif x<0.5:
        return 0
    else:
        return 0

def binary(x):
    if x>=0.5:
        return 1
    elif x<0.5:
        return 0
            

# In sample performance

In [32]:
#Portfolio Statistics
def port_stats_insample(returns,algo):
    tree_predictions = pd.read_csv("Trend Following Return Prediction/In Sample_Tree Predictions_{}.csv".format(algo))
    tree_predictions['Row.names'] = tree_predictions['Row.names']-1 #Pandas is 0 based, R is 1 based
    tree_predictions = tree_predictions.set_index(tree_predictions['Row.names'])

    tree_predictions['y'] = pd.to_datetime(tree_predictions['y'], dayfirst=True)
    tree_predictions['month'] = tree_predictions['y'].astype(str).str[:7]
    tree_predictions = tree_predictions.set_index('month').drop('y',axis=1)
    
    test = tree_predictions.merge(returns,how='left',on='month')
    test_cumlror_idx = pd.DataFrame.copy(test)
    
    #drawdown set up
    test_cumlror_idx['portfolio_CumlProd'] = (1 + test_cumlror_idx["total"]).cumprod()
    test_cumlror_idx['portfolio_RollingMax'] = test_cumlror_idx["portfolio_CumlProd"].expanding().max()
    test_cumlror_idx['portfolio_DD'] = (test_cumlror_idx['portfolio_RollingMax']-test_cumlror_idx['portfolio_CumlProd'])/test_cumlror_idx['portfolio_RollingMax']
    
    #stats
    asset_stats = pd.DataFrame(columns=['{}'.format(algo)])
    asset_stats.loc[0,'{}'.format(algo)] = test['total'].mean()*(12)
    asset_stats.loc[1,'{}'.format(algo)] = test[test['total'] != 0]['total'].std()*np.sqrt(12)
    asset_stats.loc[2,'{}'.format(algo)] = asset_stats.loc[0,'{}'.format(algo)]/asset_stats.loc[1,'{}'.format(algo)]
    asset_stats.loc[3,'{}'.format(algo)] = asset_stats.loc[0,'{}'.format(algo)]/(test[test['total']<0]['total'].std()*np.sqrt(12))
    asset_stats.loc[4,'{}'.format(algo)] = test_cumlror_idx['portfolio_DD'].max()
    asset_stats.loc[5,'{}'.format(algo)] = "NA"
    asset_stats.loc[6,'{}'.format(algo)] = "NA"

    asset_stats['Stat'] = ['Average Return','Vol','SR','Sortino Ratio','Max DD','Prob. of SR Improvement','SR p-value']
    asset_stats = asset_stats.set_index('Stat')
    return asset_stats

In [33]:
#Enhanced Portfolio Statistics
def enh_port_stats_insample(returns,algo,transformation):
    tree_predictions = pd.read_csv("Trend Following Return Prediction/In Sample_Tree Predictions_{}.csv".format(algo))
    tree_predictions['multiplier'] = tree_predictions['1'].apply(transformation)
    tree_predictions['Row.names'] = tree_predictions['Row.names']-1 #Pandas is 0 based, R is 1 based
    tree_predictions = tree_predictions.set_index(tree_predictions['Row.names'])

    tree_predictions['y'] = pd.to_datetime(tree_predictions['y'], dayfirst=True)
    tree_predictions['month'] = tree_predictions['y'].astype(str).str[:7]
    tree_predictions = tree_predictions.set_index('month').drop('y',axis=1)
    
    test = tree_predictions.merge(returns,how='left',on='month')
    test['total'] = test['multiplier']*test['total']
    test_cumlror_idx = pd.DataFrame.copy(test)
      
    #drawdown set up
    test_cumlror_idx['portfolio_CumlProd'] = (1 + test_cumlror_idx["total"]).cumprod()
    test_cumlror_idx['portfolio_RollingMax'] = test_cumlror_idx["portfolio_CumlProd"].expanding().max()
    test_cumlror_idx['portfolio_DD'] = (test_cumlror_idx['portfolio_RollingMax']-test_cumlror_idx['portfolio_CumlProd'])/test_cumlror_idx['portfolio_RollingMax']
    
    #stats
    asset_stats = pd.DataFrame(columns=['{}'.format(algo)])
    asset_stats.loc[0,'{}'.format(algo)] = test['total'].mean()*(12)
    #asset_stats.loc[1,'{}'.format(algo)] = test[test['total'] != 0]['total'].std()*np.sqrt(12) #I think best to include zeros in vol calculation so ignore this row
    asset_stats.loc[1,'{}'.format(algo)] = test['total'].std()*np.sqrt(12)
    asset_stats.loc[2,'{}'.format(algo)] = asset_stats.loc[0,'{}'.format(algo)]/asset_stats.loc[1,'{}'.format(algo)]
    asset_stats.loc[3,'{}'.format(algo)] = asset_stats.loc[0,'{}'.format(algo)]/(test[test['total']<0]['total'].std()*np.sqrt(12))
    asset_stats.loc[4,'{}'.format(algo)] = test_cumlror_idx['portfolio_DD'].max()

    #Probabilistic SR
    lamda3 = skew(test['total'])
    lamda4 = kurtosis(test['total'], fisher=False)
    SR_Monthly_Enhanced = asset_stats.loc[2,'{}'.format(algo)]*1/(np.sqrt(12))
    SR_Monthly_Regular = port_stats_insample(EQ,"EQ").loc['SR','EQ']*1/(np.sqrt(12))
    T = len(test['total'])
    A = ((SR_Monthly_Enhanced-SR_Monthly_Regular)*np.sqrt(T-1))/np.sqrt(1-(lamda3*SR_Monthly_Enhanced)+((lamda4-1)/4)*(SR_Monthly_Enhanced**2))
    Z_A = norm.cdf(A)
    pvalue = 1-Z_A
    
    asset_stats.loc[5,'{}'.format(algo)] = Z_A
    asset_stats.loc[6,'{}'.format(algo)] = pvalue
    
    asset_stats['Stat'] = ['Average Return','Vol','SR','Sortino Ratio','Max DD','Prob. of SR Improvement','SR p-value']
    asset_stats = asset_stats.set_index('Stat')
    return asset_stats

In [34]:
def tree_stats_insample(transformation):  
    finalstats = port_stats_insample(EQ,"EQ").merge(enh_port_stats_insample(EQ,"EQ",transformation),how='left',on='Stat')
    finalstats = finalstats.merge(enh_port_stats_insample(EQ,"EQ1",transformation),how='left',on='Stat')
    finalstats = finalstats.merge(enh_port_stats_insample(EQ,"EQ2",transformation),how='left',on='Stat')
    return finalstats

In [35]:
tree_stats_insample(binary)

,EQ_x,EQ_y,EQ1,EQ2
Stat,,,,
Average Return,0.159379,0.276302,0.332083,0.285372
Vol,0.261216,0.193747,0.154888,0.202806
SR,0.610143,1.4261,2.14402,1.40712
Sortino Ratio,1.02974,1.39417,7.54673,1.93633
Max DD,0.534897,0.274726,0.0453,0.33471
Prob. of SR Improvement,NA,0.996431,1,0.997681
SR p-value,NA,0.0035694,0,0.00231867


# Out of sample performance

To be used only once models have been decided by in sample tuning

In [36]:
#Portfolio Statistics
def port_stats(returns,algo):
    tree_predictions = pd.read_csv("Trend Following Return Prediction/Tree Predictions_{}.csv".format(algo))
    tree_predictions['Row.names'] = tree_predictions['Row.names']-1 #Pandas is 0 based, R is 1 based
    tree_predictions = tree_predictions.set_index(tree_predictions['Row.names'])

    tree_predictions['y'] = pd.to_datetime(tree_predictions['y'], dayfirst=True)
    tree_predictions['month'] = tree_predictions['y'].astype(str).str[:7]
    tree_predictions = tree_predictions.set_index('month').drop('y',axis=1)
    
    test = tree_predictions.merge(returns,how='left',on='month')
    test_cumlror_idx = pd.DataFrame.copy(test)
    
    #drawdown set up
    test_cumlror_idx['portfolio_CumlProd'] = (1 + test_cumlror_idx["total"]).cumprod()
    test_cumlror_idx['portfolio_RollingMax'] = test_cumlror_idx["portfolio_CumlProd"].expanding().max()
    test_cumlror_idx['portfolio_DD'] = (test_cumlror_idx['portfolio_RollingMax']-test_cumlror_idx['portfolio_CumlProd'])/test_cumlror_idx['portfolio_RollingMax']
    
    #stats
    asset_stats = pd.DataFrame(columns=['{}'.format(algo)])
    asset_stats.loc[0,'{}'.format(algo)] = test['total'].mean()*(12)
    asset_stats.loc[1,'{}'.format(algo)] = test[test['total'] != 0]['total'].std()*np.sqrt(12)
    asset_stats.loc[2,'{}'.format(algo)] = asset_stats.loc[0,'{}'.format(algo)]/asset_stats.loc[1,'{}'.format(algo)]
    asset_stats.loc[3,'{}'.format(algo)] = asset_stats.loc[0,'{}'.format(algo)]/(test[test['total']<0]['total'].std()*np.sqrt(12))
    asset_stats.loc[4,'{}'.format(algo)] = test_cumlror_idx['portfolio_DD'].max()
    asset_stats.loc[5,'{}'.format(algo)] = "NA"
    asset_stats.loc[6,'{}'.format(algo)] = "NA"

    asset_stats['Stat'] = ['Average Return','Vol','SR','Sortino Ratio','Max DD','Prob. of SR Improvement','SR p-value']
    asset_stats = asset_stats.set_index('Stat')
    test.to_csv("Portfolio Output_Original.csv.csv")
    return asset_stats

In [37]:
#Enhanced Portfolio Statistics
def enh_port_stats(returns,algo,transformation):
    tree_predictions = pd.read_csv("Trend Following Return Prediction/Tree Predictions_{}.csv".format(algo))
    tree_predictions['multiplier'] = tree_predictions['1'].apply(transformation)
    tree_predictions['Row.names'] = tree_predictions['Row.names']-1 #Pandas is 0 based, R is 1 based
    tree_predictions = tree_predictions.set_index(tree_predictions['Row.names'])

    tree_predictions['y'] = pd.to_datetime(tree_predictions['y'], dayfirst=True)
    tree_predictions['month'] = tree_predictions['y'].astype(str).str[:7]
    tree_predictions = tree_predictions.set_index('month').drop('y',axis=1)
    
    test = tree_predictions.merge(returns,how='left',on='month')
    test['total'] = test['multiplier']*test['total']
    test_cumlror_idx = pd.DataFrame.copy(test)
      
    #drawdown set up
    test_cumlror_idx['portfolio_CumlProd'] = (1 + test_cumlror_idx["total"]).cumprod()
    test_cumlror_idx['portfolio_RollingMax'] = test_cumlror_idx["portfolio_CumlProd"].expanding().max()
    test_cumlror_idx['portfolio_DD'] = (test_cumlror_idx['portfolio_RollingMax']-test_cumlror_idx['portfolio_CumlProd'])/test_cumlror_idx['portfolio_RollingMax']
    
    #stats
    asset_stats = pd.DataFrame(columns=['{}'.format(algo)])
    asset_stats.loc[0,'{}'.format(algo)] = test['total'].mean()*(12)
    #asset_stats.loc[1,'{}'.format(algo)] = test[test['total'] != 0]['total'].std()*np.sqrt(12) #I think best to include zeros in vol calculation so ignore this row
    asset_stats.loc[1,'{}'.format(algo)] = test['total'].std()*np.sqrt(12)
    asset_stats.loc[2,'{}'.format(algo)] = asset_stats.loc[0,'{}'.format(algo)]/asset_stats.loc[1,'{}'.format(algo)]
    asset_stats.loc[3,'{}'.format(algo)] = asset_stats.loc[0,'{}'.format(algo)]/(test[test['total']<0]['total'].std()*np.sqrt(12))
    asset_stats.loc[4,'{}'.format(algo)] = test_cumlror_idx['portfolio_DD'].max()

    #Probabilistic SR
    lamda3 = skew(test['total'])
    lamda4 = kurtosis(test['total'], fisher=False)
    SR_Monthly_Enhanced = asset_stats.loc[2,'{}'.format(algo)]*1/(np.sqrt(12))
    SR_Monthly_Regular = port_stats(EQ,"EQ").loc['SR','EQ']*1/(np.sqrt(12))
    T = len(test['total'])
    A = ((SR_Monthly_Enhanced-SR_Monthly_Regular)*np.sqrt(T-1))/np.sqrt(1-(lamda3*SR_Monthly_Enhanced)+((lamda4-1)/4)*(SR_Monthly_Enhanced**2))
    Z_A = norm.cdf(A)
    pvalue = 1-Z_A
    
    asset_stats.loc[5,'{}'.format(algo)] = Z_A
    asset_stats.loc[6,'{}'.format(algo)] = pvalue
    
    asset_stats['Stat'] = ['Average Return','Vol','SR','Sortino Ratio','Max DD','Prob. of SR Improvement','SR p-value']
    asset_stats = asset_stats.set_index('Stat')
    test.to_csv("Portfolio Output_{}.csv".format(algo))
    return asset_stats

In [38]:
#adjust according to model selected from in sample tuning
def final_stats(transformation):  
    finalstats = port_stats(EQ,"EQ").merge(enh_port_stats(EQ,"EQ_Tree",transformation),how='left',on='Stat')
    finalstats = finalstats.merge(enh_port_stats(EQ,"EQ_RF",transformation),how='left',on='Stat')
    finalstats = finalstats.merge(enh_port_stats(EQ,"EQ_AB",transformation),how='left',on='Stat')
    finalstats = finalstats.merge(enh_port_stats(EQ,"EQ_Bag",transformation),how='left',on='Stat')
    finalstats = finalstats.merge(enh_port_stats(EQ,"EQ_NNet",transformation),how='left',on='Stat')
    return finalstats

In [39]:
final_stats(binary)

,EQ,EQ_Tree,EQ_RF,EQ_AB,EQ_Bag,EQ_NNet
Stat,,,,,,
Average Return,-0.0492857,0.0182743,0.09168,0.0311143,0.0753429,0.0178457
Vol,0.269631,0.16299,0.100797,0.158488,0.17065,0.173061
SR,-0.18279,0.112119,0.909553,0.19632,0.441505,0.103118
Sortino Ratio,-0.195459,0.0710516,0.891071,0.171451,0.413221,0.0640507
Max DD,0.719069,0.312132,0.0855,0.241156,0.243669,0.268
Prob. of SR Improvement,NA,0.752233,0.997599,0.808437,0.91695,0.745757
SR p-value,NA,0.247767,0.00240089,0.191563,0.0830505,0.254243


In [61]:
final_stats(binary).to_csv("Performance Metrics Table.csv")